# A simple chi analysis

In this example we will grab some data for a simple chi analysis. First, import the relevent modules. 

In [1]:
import lsdviztools
lsdviztools.__version__

'0.4.6'

In [4]:
pip install lsdviztools --upgrade

     |████████████████████████████████| 268 kB 3.3 MB/s eta 0:00:01
  Attempting uninstall: lsdviztools
    Found existing installation: lsdviztools 0.4.4
    Uninstalling lsdviztools-0.4.4:
      Successfully uninstalled lsdviztools-0.4.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

## Part 1: Grab some data

We will grab some data from the Shillong Plateau in India. I use google maps to get the latitude and longitude of the southwest and northeast corners. Once you give the `ot_scraper` the corners and the type of data to download (in this case SRTM30 data), you can move on to downloading.

In [ ]:
SH_DEM = bmt.ot_scraper(source = "SRTM30",
                        longitude_W = 91.271109, 
                        longitude_E = 92.214265, 
                        latitude_S = 25.107807, 
                        latitude_N = 25.627673,
                        prefix = "shillong")
SH_DEM.print_parameters()

Now call the download. You need to wait until the `*` symbol dissappears from the right of the line (when it stops saying `In[*]`), that indicates the download has finished. 

In [ ]:
fullfname,path,fname = SH_DEM.download_pythonic()

Okay, now we need to do two things. First, we need to convert to UTM. Then we need to convert to ENVI bil format. 

In [ ]:
SH_DEM.to_UTM_pythonic()
RasterFile = "shillong_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

Okay, we now have an ENVI bil DEM. We are ready to start doing topographic analysis!

## Part 2, get some basin outlets

I want to extract some specific basins. I will get the lat-long of their outlets to select them (use *What's here* on google maps. I then create a csv containing the latitude and longitude using pandas. 

In [ ]:
import pandas as pd

In [ ]:
d = {'latitude': [25.187265,25.178305,25.176887,25.183027], 
     'longitude': [92.016604,91.756626,91.633770,91.378109]}
df = pd.DataFrame(data=d)
print(df)

In [ ]:
df.to_csv("basin_outlets.csv",index=False)

## Part 3: Running the analysis

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true", 
                    "get_basins_from_outlets" : "true", 
                    "basin_outlet_csv" : "basin_outlets.csv", 
                    "search_radius_nodes" : "15",
                    "convert_csv_to_geojson" : "true",
                    "print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}

In [ ]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "shillong_SRTM30_UTM",
                                 write_prefix= "shillong_SRTM30_UTM",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

## Part 4: Plotting the data

First, lets just see where we are.

In [ ]:
DataDirectory = "./"
Base_file = "shillong_SRTM30_UTM"  
fig = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False)

In [ ]:
fig.show()

<img src="shillong_SRTM30_UTM_hillshade.png" width=600>

Now plot the channels

In [ ]:
ChannelFileName = "shillong_SRTM30_UTM_chi_data_map.csv"
channel_img = lsdmw.PrintChannels(DataDirectory,Base_file,ChannelFileName, 
                               cmap = "gist_earth",
                               size_format = "geomorphology", dpi = 300, 
                               plotting_column = "basin_key", save_fig = False)

In [ ]:
channel_img.show()

<img src="shillong_SRTM30_UTM_channels_coloured_by_basin.png" width=600>

Now show the basins and the channels

In [ ]:
basin_img = lsdmw.PrintChannelsAndBasins(DataDirectory,Base_file,
                                              add_basin_labels = True, 
                                              cmap = "jet",  
                                              size_format = "geomorphology", dpi = 300, save_fig=False)

In [ ]:
basin_img.show()

<img src="shillong_SRTM30_UTM_channels_with_basins.png" width=800>

In [ ]:
basin2_img_name = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,
                                           use_keys_not_junctions = True, 
                                           show_colourbar = False,cmap = "jet", 
                                           colorbarlabel = "colourbar", size_format = "geomorphology",
                                           fig_format = "png", dpi = 250, 
                                           include_channels = False, label_basins = True)

<img src="shillong_SRTM30_UTM_selected_basins.png" width=800>

In [ ]:
basin2_img_name = lsdmw.SimpleProfile(DataDirectory,Base_file, ChannelFileName, cmap = "jet", size_format = "ESURF", fig_format = "png", dpi = 250, basin_key=[0], source_key=[0])